In [ ]:
from pathlib import Path
from collections import deque
import json
import re
from slpp import slpp as lua
from dataclasses import dataclass, field
from graphviz import Digraph
import networkx as nx
import tkinter as tk
import pygraphviz as pgv

In [ ]:
text = Path("data/recipe.lua").read_text()

# Remove wrapper
text = text.removeprefix("data:extend(\n").removesuffix(")\n")
text = text.strip()

# Remove comments
text = re.sub(r"--.*", "", text)

raw_recipes = lua.decode(text)

Path("data/recipe.json").write_text(json.dumps(raw_recipes, indent=2))

raw_recipe_keys = raw_recipe_keys = {key for recipe in raw_recipes for key in recipe.keys()}

print(len(raw_recipes))
print(raw_recipe_keys)

In [ ]:
@dataclass
class Recipe:
    name: str
    ingredients: dict[str, int] = field(default_factory=dict)
    energy_required: float = 0.5
    result_count: float = 1

DIFFICULTY = "normal"
recipes: dict[str, Recipe] = {}
for raw_recipe in raw_recipes:
    name = raw_recipe["name"]
    amount_dict = raw_recipe.get(DIFFICULTY, raw_recipe)

    # Parse out the ingredients
    ingredients: dict[str, int] = {}
    raw_ingredients = amount_dict.get("ingredients")
    for raw_ingredient in raw_ingredients:
        if isinstance(raw_ingredient, list):
            ingredient_name, ingredient_amount = raw_ingredient
        elif isinstance(raw_ingredient, dict):
            ingredient_name = raw_ingredient.get("name")
            ingredient_amount = raw_ingredient.get("amount")
        else:
            print(f"Issue with ingredients for `{name}`")
            continue
        ingredients[ingredient_name] = ingredient_amount

    # Parse out the results
    raw_result = amount_dict.get("result")
    if raw_result is None:
        print(f"Skipping {name}")
        continue
    if not isinstance(raw_result, str) or raw_result != name:
        print(raw_result, name, "is weird, will skip")
        continue

    result_count = amount_dict.get("result_count", 1)
    # # This logic is for fluid processing stuff, don't care about it right now 
    # elif raw_result is None and isinstance(raw_results, list):
    #     for raw_result in raw_results:
    #         if isinstance(raw_result, list):
    #             result_name, result_amount = raw_result
    #         elif isinstance(raw_result, dict):
    #             result_name = raw_result.get("name")
    #             result_amount = raw_result.get("amount")
    #             raw_probability = raw_result.get("probability")
    #             if raw_probability is not None:
    #                 result_amount = result_amount * raw_probability
    #         else:
    #             print(f"Issue with results for `{name}`")
    #             continue
    #         results[result_name] = result_amount

    recipes[name] = Recipe(
        name=name,
        ingredients=ingredients,
        energy_required=raw_recipe.get("energy_required", 0.5),
        result_count=result_count,
    )

# Double check that ingredients and results are parsed correctly
for recipe_name, recipe in recipes.items():
    if recipe.ingredients == {}:
        print(f"Issue with ingredients for `{recipe_name}`")


In [ ]:
RAW_MATERIALS = {
    "iron-plate",
    "copper-plate",
    "water",
    "petroleum-gas",
    "coal",
    "stone",
    "wood",
}

def build_dependency_edges(recipes, target, raw_materials=RAW_MATERIALS):
    target = [target] if isinstance(target, str) else target
    visited = set()
    edges = []
    queue = deque(target)

    while queue:
        product = queue.popleft()
        if product in visited:
            continue
        visited.add(product)

        # Find recipes that produce this product
        recipe = recipes.get[product]
        if recipe is None or product in raw_materials:
            # Skip product without recipes, aren't actually products
            continue
        for ingredient in recipe.ingredients.keys():
            if (ingredient, product) not in edges:
                edges.append((ingredient, product))
            if ingredient not in visited:
                queue.append(ingredient)
    
    return edges

edges = build_dependency_edges(recipes, ["car"])  # spidertron

dot = Digraph(comment="Factorio Recipes", engine="dot")
for edge in edges:
    dot.edge(edge[0], edge[1])

dot.render("recipes", format="png", view=True)

In [ ]:


def build_dependency_graph(recipes: dict[str, Recipe], target: str, target_rate: float = 1.0):
    """
    Build a graph for a given target item with demand labels (units/second).
    target_rate = how many target items per second you want.
    """
    dot = Digraph(comment="Factorio Recipes", engine="dot")

    # track demand
    demand = {target: target_rate}
    queue = deque([target])

    while queue:
        product = queue.popleft()
        if product in RAW_MATERIALS:
            continue

        # Find recipe that makes this product
        recipe = next((r for r in recipes.values() if product in r.results), None)
        if recipe is None:
            continue

        product_amount = recipe.results[product]
        craft_time = recipe.energy_required or 0.5
        product_rate = product_amount / craft_time  # items/s per machine

        # scale factor: how many crafts/sec to satisfy current demand
        crafts_per_sec = demand[product] / product_rate

        for ing, ing_amount in recipe.ingredients.items():
            ing_rate = ing_amount * crafts_per_sec
            demand[ing] = demand.get(ing, 0) + ing_rate

            # add edge with demand label
            dot.edge(ing, product, label=f"{ing_rate:.2f}/s")

            if ing not in RAW_MATERIALS:
                queue.append(ing)

    return dot, demand

dot, demand = build_dependency_graph(recipes, "cart", target_rate=0.1)

# Render graph
dot.render("recipes", format="png", view=True)

# Print total demand table
for item, rate in demand.items():
    print(f"{item}: {rate:.2f}/s")


# TK

In [ ]:
# Scale positions to canvas
def scale(x, y, width, height, margin=50):
    # Graphviz has origin top-left, may need flip y
    # Find min/max for scaling
    xs = [p[0] for p in pos.values()]
    ys = [p[1] for p in pos.values()]
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    
    sx = margin + (x - min_x) / (max_x - min_x) * (width - 2*margin)
    sy = margin + (y - min_y) / (max_y - min_y) * (height - 2*margin)
    return sx, sy


A = pgv.AGraph(directed=True)
A.add_edges_from(edges)
A.layout(prog='dot')

A

# root = tk.Tk()
# root.title("Factorio Recipe Graph")
# canvas = tk.Canvas(root, width=800, height=600, bg="white")
# canvas.pack()

# # Example interaction: print node name on click
# def on_click(event):
#     clicked_items = canvas.find_closest(event.x, event.y)
#     for node, nid in node_ids.items():
#         if nid in clicked_items:
#             print(f"Clicked node: {node}")

# canvas.bind("<Button-1>", on_click)

# root.mainloop()

In [ ]:
A.edges()[0].attrs['pos']